In [1]:
# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# CNN으로 MNIST 분류하기

## 모델 이해하기
- 합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 다름

**1. 첫 번째 표기 방법**

합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명

**2. 두 번째 표기 방법**

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 봄



- 즉, pooling도 하나의 층으로 보느냐의 문제인데 누가 옳고 틀리냐의 문제는 아니므로 이번 챕터에서는 편의를 위해 두 번째 표기 방법 채택함
- 모델의 아키텍처는 총 3개의 층으로 구성됨

In [ ]:
# 1번 layer : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU
맥스풀링(kernel_size = 2, stride = 2)

# 2번 layer : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size = 3, stride = 1, padding = 1) + 활성화 함수 ReLU
맥스풀링(kernel_size = 2, stride = 2)

# 3번 layer : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size x 7 x 7 x 64 -> batch_size x 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax

## 모델 구현하기

In [4]:
# 1. 필요한 도구 import와 입력의 정의
import torch
import torch.nn as nn

# (batch_size, channel, height, width) 크기의 텐서 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [8]:
# 2. 합성곱층과 풀링 선언하기
# (c, h, w) = (1, 28, 28)의 데이터가 1만큼 padding을 한 후 (3, 3) 필터와 convolution을 함
# O_h = (28 - 3 + 1*2)//1 + 1 = 28 / O_w = (28 - 3 + 1*2)//1 + 1 = 28
# 특성맵의 채널 수(C_0)는 nn.Conv2d()의 in_channel, out_channel로 조절하는 듯

conv1 = nn.Conv2d(1, 32, 3, padding=1) # in_channel, out_channel, kernel_size, padding
print(conv1)

conv2 = nn.Conv2d(32, 64, 3, padding=1) # in_channel, out_channel, kernel_size, padding
print(conv2)

pool = nn.MaxPool2d(2) # 정수 하나를 인자로 넣으면 kernel_size, stride가 둘 다 해당 값으로 지정됨
print(pool)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [13]:
# 3. 구현체를 연결하여 모델 만들기
# 입력(1, 1, 28, 28)을 첫 번째 합성곱층에 통과시키고 난 후의 Tensor의 크기
out = conv1(inputs)
print(out.shape) # (batch_size, channel, height, width)

# maxpooling에 통과시키고 난 후의 Tensor의 크기
out = pool(out)
print(out.shape)

# 이를 다시 두 번째 합성곱층에 통과시키기
out = conv2(out)
print(out.shape) # 예상 : (1, 64, 14, 14)

# 이를 maxpooling에 통과시키기
out = pool(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [14]:
# .size(n) : 텐서의 n번째 차원에 접근하게 해줌
# .view() : reshape 해주는 기능
# 첫 번째 차원인 배치 차원은 그대로 두고 나머지 펼치기
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [15]:
# 배치 차원을 제외하고 모두 하나의 차원으로 통합됨
# 이에 대해 전결합층(Fully-Connected layer)을 통과시키기
# 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환함
fc = nn.Linear(3136, 10) # i=(input_dim, output_dim)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## CNN으로 MNIST 분류하기

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니면 cpu가 됨

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [24]:
# 학습에 사용할 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# dataloader를 사용하여 데이터를 다루기 위해서 데이터셋 정의
# 아직 dataloader 사용 안 한 듯..
mnist_train = dsets.MNIST(root = 'MNIST_data/', # 다운로드 경로 지정
                         train = True, # True를 지정하면 train 데이터로 다운로드
                         transform = transforms.ToTensor(), # 텐서로 변환
                         download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', # 다운로드 경로 지정
                         train = False, # False를 지정하면 test 데이터로 다운로드
                         transform = transforms.ToTensor(), # 텐서로 변환
                         download = True)

# dataloader를 사용하여 batch_size를 지정해줌
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True) # 데이터를 batch_size로 나누었을 때 나머지를 버림

In [25]:
# 클래스로 모델 설계
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 1번째 층
        # ImgIn shape = (?, 28, 28, 1) = (batch_size, h, w, c) 인듯?
        # Conv -> (?, 28, 28, 32)
        # Pool -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 2번째 층
        # ImgIn shape = (?, 14, 14, 32)
        # Conv -> (?, 14, 14, 64)
        # Pool -> (?, 7, 7, 64)   
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 전결합층 7 x 7 x 64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해서 flatten
        out = self.fc(out)
        return out

In [30]:
# CNN 모델 정의하기
model = CNN().to(device)

# cost function과 optimizer 정의하기
criterion = torch.nn.CrossEntropyLoss().to(device) # 손실 함수에 softmax 함수 포함되어 있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
# 총 배치의 수
# batch_size = 100으로 했으므로 train data의 총 개수가 60,000개임
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [58]:
# 모델 훈련시키기 -> test_acc : 98.9
# epoch : 전체 트레이닝 셋(60,000개의 데이터)이 순전파, 역전파를 통해 신경망을 통과한 횟수

for epoch in range(training_epochs): # 전체 학습 과정을 15번 반복함
    avg_cost = 0
    
    for X, Y in data_loader: # mini batch 단위로 꺼내옴. X는 mini batch, Y는 label
        # X.shape = (28, 28)
        # Y : not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() 
        # 미분을 통해 얻은 기울기를 0으로 초기화
        # 기울기를 초기화하여야만 새로운 w, b에 대한 새로운 기울기를 구할 수 있음 
        # PyTorch에서는 gradients값들을 추후 backward를 해줄 때 계속 더해주기 때문에
        # 이상적으로 학습이 이루어지려면 한 번의 학습이 완료되어지면 gradients를 항상 0으로 만들어줘야 함
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        # 손실 함수를 미분하여 w, b에 대한 gradient 계산
        cost.backward() 
        
        # 인수로 들어갔던 w, b에서 리턴되는 변수들의 기울기에 learning_rate를 곱하여 빼줌으로써 업데이트
        optimizer.step() 

        avg_cost += cost / total_batch # total_batch : 600 / total_batch를 나중에 나눠줘도 되지 않음?
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    # {:>4} : 4자리 오른쪽 정렬
    # {:>.9} : 소수점 9자리 오른쪽 정렬

[Epoch:    1] cost = 0.00537449541
[Epoch:    2] cost = 0.00516238436
[Epoch:    3] cost = 0.00134891795
[Epoch:    4] cost = 0.000296522252
[Epoch:    5] cost = 3.83469051e-05
[Epoch:    6] cost = 1.71281372e-05
[Epoch:    7] cost = 1.27903522e-05
[Epoch:    8] cost = 9.87666954e-06
[Epoch:    9] cost = 7.80600931e-06
[Epoch:   10] cost = 6.2441386e-06
[Epoch:   11] cost = 4.98965755e-06
[Epoch:   12] cost = 4.03938338e-06
[Epoch:   13] cost = 3.18378625e-06
[Epoch:   14] cost = 2.48596734e-06
[Epoch:   15] cost = 1.96097631e-06


In [51]:
# 모델 훈련시키기 (total_batch를 나중에 나눠준 ver.) -> test_acc : 98.9
# epoch : 전체 트레이닝 셋(60,000개의 데이터)이 순전파, 역전파를 통해 신경망을 통과한 횟수

for epoch in range(training_epochs): # 전체 학습 과정을 15번 반복함
    
    cost_sum = 0
    
    for X, Y in data_loader: # mini batch 단위로 꺼내옴. X는 mini batch, Y는 label
        # X.shape = (28, 28)
        # Y : not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() 
        # 미분을 통해 얻은 기울기를 0으로 초기화
        # 기울기를 초기화하여야만 새로운 w, b에 대한 새로운 기울기를 구할 수 있음 
        # PyTorch에서는 gradients값들을 추후 backward를 해줄 때 계속 더해주기 때문에
        # 이상적으로 학습이 이루어지려면 한 번의 학습이 완료되어지면 gradients를 항상 0으로 만들어줘야 함
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        # 손실 함수를 미분하여 w, b에 대한 gradient 계산
        cost.backward() 
        
        # 인수로 들어갔던 w, b에서 리턴되는 변수들의 기울기에 learning_rate를 곱하여 빼줌으로써 업데이트
        optimizer.step() 

        cost_sum += cost 
    
    avg_cost = cost_sum / total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    # {:>4} : 4자리 오른쪽 정렬
    # {:>.9} : 소수점 9자리 오른쪽 정렬

[Epoch:    1] cost = 0.00319483457
[Epoch:    2] cost = 0.00121779973
[Epoch:    3] cost = 0.00222596992
[Epoch:    4] cost = 0.00187778776
[Epoch:    5] cost = 0.000289090327
[Epoch:    6] cost = 4.45629303e-05
[Epoch:    7] cost = 2.85160913e-05
[Epoch:    8] cost = 1.49460056e-05
[Epoch:    9] cost = 1.16581878e-05
[Epoch:   10] cost = 9.01890689e-06
[Epoch:   11] cost = 7.22428331e-06
[Epoch:   12] cost = 5.73538728e-06
[Epoch:   13] cost = 4.62762409e-06
[Epoch:   14] cost = 3.75624813e-06
[Epoch:   15] cost = 2.82945666e-06


In [60]:
# 테스트 해보기
# 학습을 진행하지 않을 것이므로 torch.no_grad()

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    print('1. prediction.shape -- ',prediction.shape)
    
    # 각 배치별로 가장 높은 가능성의 숫자 클래스를 뽑아서 정답과 맞는지 확인 후 boolean값으로 저장
    correct_prediction = torch.argmax(prediction, dim=1) == Y_test
    print('2. type(correct_prediction) -- ', type(correct_prediction))
    print('3. correct_prediction.shape -- ', correct_prediction.shape)
    
    # boolean -> float 형 변환 후 평균 구하기
    accuracy = correct_prediction.float().mean()
    print('4. accuracy -- ', accuracy)
    print('5. accuracy.item() -- ', accuracy.item())

1. prediction.shape --  torch.Size([10000, 10])
2. type(correct_prediction) --  <class 'torch.Tensor'>
3. correct_prediction.shape --  torch.Size([10000])
4. accuracy --  tensor(0.9894)
5. accuracy.item() --  0.9894000291824341


# 깊은 CNN으로 MNIST 분류하기
- 앞서 배운 CNN에 층을 더 추가하여 MNIST를 분류할 것임

## 모델 이해하기
- 모델의 아키텍처는 총 5개의 층으로 구성됨
- 앞서 배운 챕터에서 1번 레이어와 2번 레이어는 동일하되, 새로운 합성곱층과 전결합층을 추가함

In [ ]:
# 1번 layer : 합성곱층(Convolutional layer)
nn.Conv2d(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1) + ReLU
MaxPool2d(kernel_size = 2, stride = 2)

# 2번 layer : 합성곱층(Convolutional layer)
nn.Conv2d(in_channel = 32, out_channel = 64, kernel_size = 3, stride = 1, padding = 1) + ReLU
MaxPool2d(kernel_size = 2, stride = 2)

# 3번 layer : 합성곱층(Convolutional layer)
nn.Conv2d(in_channel = 64, out_channel = 128, kernel_size = 3, stride = 1, padding = 1) + ReLU
MaxPool2d(kernel_size = 2, stride = 2, padding = 1)

# 4번 layer : 전결합층(Fully-Connected layer)
특성맵을 펼침 # batch_size * 4 * 4 * 128 -> batch_size * 2048
FC(뉴런 625개) + ReLU

# 5번 layer : 전결합층(Fully-Connected layer)
FC(뉴런 10개) + Softmax

## 깊은 CNN으로 MNIST 분류하기

In [59]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [62]:
# 학습에 사용할 parameter 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

# dataloader를 사용하여 batch_size 지정
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

# 클래스로 모델 설계
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        '''
        L1 ImgIn shape = (?, 28, 28, 1)
        Conv = (?, 28, 28, 32) = (batch_size, h, w, c)
        Pool = (?, 14, 14, 32) = (batch_size, h, w, c)
        '''
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1), # in_channels, out_channels
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        '''
        L2 ImgIn shape = (?, 14, 14, 32)
        Conv = (?, 14, 14, 64) = (batch_size, h, w, c)
        Pool = (?, 7, 7, 64) = (batch_size, h, w, c)        
        '''
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        '''
        L3 ImgIn shape = (?, 7, 7, 64)
        Conv = (?, 7, 7, 128) = (batch_size, h, w, c)
        Pool = (?, 4, 4, 128) = (batch_size, h, w, c)                
        '''
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
       
        '''
        L4 FC 4x4x128 inputs -> 625 outputs (625 from where???)
        '''
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1 - self.keep_prob))
        
        '''
        L5 Final FC 625 inputs -> 10 outputs
        '''
        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # flatten for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [ ]:
# CNN 모델 정의
model = CNN().to(device)

# 손실함수와 optimizer 정의
criterion = torch.nn.CrossEntropyLoss().to(device) # 손실 함수에 softmax 함수 포함되어 있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('총 batch의 수 : {}'.format(total_batch))

# 모델 훈련시키기
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X) # 변수명 왜 hypothesis???
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

총 batch의 수 : 600
[Epoch:    1] cost = 0.198703051
[Epoch:    2] cost = 0.0517416932


In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())